In [122]:
import pandas as pd
# imports googlesearch to use requests and BS4 to scrape google
from googlesearch import search
import csv

In [123]:
# In the future, we can improve this by being able to read in any/all the datasets

# reads in the data set
#data = pd.read_csv(r"Affiliation_Parsing/NeurIPS affiliations/NeurIPSaffiliations-2010-affiliation.csv")
openfile =(r"Affiliation_Parsing/NeurIPS affiliations gender/NeurIPS-2010/xaa.csv")
data = pd.read_csv(openfile)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Unnamed: 0                19 non-null     int64 
 1   Unnamed: 0.1              19 non-null     int64 
 2   first-author              19 non-null     object
 3   first-author-affiliation  19 non-null     object
 4   last-author               19 non-null     object
 5   last-author-affiliation   19 non-null     object
 6   title                     19 non-null     object
dtypes: int64(2), object(5)
memory usage: 1.2+ KB


In [124]:
# drops every column that is not the authors or their affiliations
data = data.drop(data.columns[0:2], axis=1)
data = data.drop(data.columns[4:5], axis=1)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   first-author              19 non-null     object
 1   first-author-affiliation  19 non-null     object
 2   last-author               19 non-null     object
 3   last-author-affiliation   19 non-null     object
dtypes: object(4)
memory usage: 736.0+ bytes


In [125]:
# imports BS4 libraries for webscraping
from re import I
import requests
from bs4 import BeautifulSoup


In [126]:
# Need to add in they/them
# Improve search


def findGender(urlLink):
    gender = ''
    ## put a url where the desciption of the author is avilable
    
    url = urlLink
   
    htmlText = requests.get(url).text
    #print(htmlText)
    soup = BeautifulSoup(htmlText, 'html.parser')
    
    
    # gathers all the p tags from the html
    soupstring=str(soup.find_all("div")) + str(soup.find_all("p"))
    #print(soupstring)
    shetags = [" she.", " she, "," she ", " She ","She ", 
              " her.", " her, "," her "," Her ","Her "
              " her."," hers, "," hers ","Hers "," Hers"
               , "She’s ", " she's " ]
    
    hetags = [" he.", " he, "," he ", " He ","He ", 
              " his.", " his, "," his "," His ","His "
              " him."," him, "," him "," Him ", "He’s ", " he's " ]
    
    theytags = [" they ", "They ", " them ", "Them " ]

    ##checks if male or female pronouns are present in the desciption

    containsfemale = any(femalepronouns in soupstring for femalepronouns in shetags)
    containsmale = any(malepronouns in soupstring for malepronouns in hetags)
    containsthey = any(theypronouns in soupstring for theypronouns in theytags)

    if containsfemale:
        gender = "female"
    elif containsmale:
        
        gender = "male"
#     elif containsthey:
#         gender = "they/them"
    else:
        gender = "inconclusive"
    return  gender 



In [127]:
# returns whether or not a substring exists in a string
def substring_exists(string, sub_string):
    return string.find(sub_string)


In [128]:
# returns a list of the first-authors
def getFirstAuthorList():
    for column in data[['first-author']]:
        authors = data[column]

        '''
        Prints out column name and its values
        '''
        #print('Column Name: '+column)
        # print(authors.values)

    lst_authors = authors.values
    return lst_authors


In [129]:
def generateLinks():

    counter = 0
    linkArray = []
    lst = getFirstAuthorList()
    print(lst)
    getAllLinks=[]
    
    #if we have any particular domains which we dont want to include in out searches we can input them in this list
    unwantedLinks = ["pdf", "dblp"]
    for person in lst:

        query = person+" " + data['first-author-affiliation'].values[counter]+" .edu"
        #print(query)
        for link in search(query, num=10, stop=10):
            # appends .edu and .org links into the array
            if(substring_exists(link, ".edu") != -1 or substring_exists(link, ".org") != -1 ):
                #checks if each link contains anything from the list of unwanted list
                containsUnwanted = any(elements in link for elements in unwantedLinks)
                if(containsUnwanted == False):
                    linkArray.append(link)

  
        getAllLinks.append(linkArray)
        linkArray=[]
        counter += 1

        with open('newCSV.csv', 'w') as file:
            write = csv.writer(file)
            write.writerow(["Author Links"])
            for val in getAllLinks:
                write.writerow([val])
        with open('newCSV.csv','r') as file1, open(openfile,'r') as file2,open('combined.csv', 'w') as result:
            file1reader = csv.reader(file1)
            file2reader = csv.reader(file2)
            result = csv.writer(result)
            result.writerows(x + y for x, y in zip(file2reader,file1reader))

        

In [136]:

def firstAuthorGenderAuthor():

    fileNew = open("combined.csv", "r")

   
    update = []
    reader = csv.DictReader(fileNew)

    for rowVal in reader:
        for x in [rowVal["Author Links"]]:
            authorList = x.split("'")
            authorList = list(filter(('[').__ne__, authorList))
            authorList = list(filter((", ").__ne__, authorList))
            authorList = list(filter(("]").__ne__, authorList))
            print(authorList)
            for link in authorList:
                if (link == '[]'):
                    break
                print('THE LINK: '+ link)
                finalGender = findGender(link)
                if(finalGender == 'male' or finalGender == 'female'):
                    break
        row = {"first-author": rowVal["first-author"],"first-author-affiliation":rowVal["first-author-affiliation"],"Gender": finalGender}
        update.append(row)
    fileNew.close()
    op = open("xaaFinal.csv", "w", newline='')
    headers = ['first-author', 'first-author-affiliation', 'Gender']
    data = csv.DictWriter(op, delimiter=',', fieldnames=headers)
    data.writerow(dict((heads, heads) for heads in headers))
    data.writerows(update)
    op.close()
    

    

In [131]:
generateLinks()

['Surya Ganguli' 'Hal Daumé III' 'Akshat Kumar' 'Alan Fern' 'Deqing Sun'
 'J. Zico Kolter' 'Ling Huang' 'Martin A Zinkevich' 'Abdeslam Boularias'
 'Amir-massoud Farahmand' 'Paul L Ruvolo' 'Peter B Jones'
 'Ulrike von Luxburg' 'HongJing Lu' 'Novi Quadrianto' 'Issei Sato'
 'Jose G Puertas' 'David Silver' 'Mohammad J Saberian']


In [138]:
firstAuthorGender()

['https://ganguli-gang.stanford.edu/', 'https://ganguli-gang.stanford.edu/people.html', 'https://ganguli-gang.stanford.edu/pubs.html', 'https://ganguli-gang.stanford.edu/teaching.html', 'https://ganguli-gang.stanford.edu/surya.html', 'https://profiles.stanford.edu/surya-ganguli', 'https://en.wikipedia.org/wiki/Surya_Ganguli', 'https://www.simonsfoundation.org/people/surya-ganguli/']
THE LINK: https://ganguli-gang.stanford.edu/
THE LINK: https://ganguli-gang.stanford.edu/people.html
THE LINK: https://ganguli-gang.stanford.edu/pubs.html
THE LINK: https://ganguli-gang.stanford.edu/teaching.html
THE LINK: https://ganguli-gang.stanford.edu/surya.html
THE LINK: https://profiles.stanford.edu/surya-ganguli
THE LINK: https://en.wikipedia.org/wiki/Surya_Ganguli
['http://www.umiacs.umd.edu/~hal/', 'https://www.cs.umd.edu/people/hal3']
THE LINK: http://www.umiacs.umd.edu/~hal/
['https://scholarworks.umass.edu/open_access_dissertations/734/', 'https://www.cics.umass.edu/news/akshat-kumar-wins-icaps